In [1]:
import yake
import sys
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import requests
import time
from IPy import IP
from collections import defaultdict
import itertools  #used to slice a dictionary

import json
import pickle

from pprint import pprint

In [2]:
#Functions used for extraction of text from the url provided

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def text_from_title(body):
    soup = BeautifulSoup(body, 'html.parser')
    title_text = soup.find('title').text
    return title_text

In [3]:
def keywordExtractor(url):
    url = url.strip()
    headers={'User-Agent': 'Mozilla/5.0'}
    resp = requests.get(url, headers=headers, timeout=3) 
    if resp.status_code >= 300:
        return {}
    html = resp.text
    text = text_from_html(html)
    title_text = text_from_title(html).lower()
    max_ngram_size = 1
    simple_kwextractor = yake.KeywordExtractor(lan="en", n = max_ngram_size, dedupLim=0.9, dedupFunc='seqm', windowsSize=1, top=20, features=None)
    keywords = simple_kwextractor.extract_keywords(text)
    title_keywords = simple_kwextractor.extract_keywords(title_text)
    title_words = dict([(t[1], t[0]) for t in title_keywords])  #to swap the term and its score to get {term:score}
    title_words = title_words.keys()
    
    keywords2 = []
    for kw in keywords:
        #since the order of term and score is reversed
        if kw[1] in title_words:
            kw2 = (kw[0]/4, kw[1])
        else:
            kw2 = kw
        keywords2.append(kw2)   #should be inside for loop so that all words are appended
    keywords2 = dict([(t[1], t[0]) for t in keywords2]) #to make a dictionary(as given in description) and also to reverse the order to {term:score}
    return keywords2

In [5]:
# load new history created since last update

path_history = './data/history.data'

try:
    with open(path_history, 'rb') as f:
        history_dict = pickle.load(f)
except:
    history_dict = {}
    
pprint(history_dict)
    
# looks like: 
# {
#     'http://localhost:8888/lab': {'scroll_dist': 0, 'time_spent': 27},
    
#     'https://codeforces.com/': {'scroll_dist': 1187.199951171875, 'time_spent': 6},
    
#     'https://github.com/kanishk509/Multi-dimensional-graph-based-search-customization': {'scroll_dist': 0,
#                                                                                       'time_spent': 8},
    
#     'https://stackoverflow.com/questions/2260446/how-to-iterate-through-dictionary-and-change-values': {'scroll_dist': 0,
#                                                                                                      'time_spent': 3},
    
#     'https://web.whatsapp.com/': {'scroll_dist': 0, 'time_spent': 4}
# }

{'https://bleacherreport.com/tennis': {'scroll_dist': 0, 'time_spent': 2},
 'https://networkx.github.io/documentation/stable/reference/algorithms/bipartite.html': {'scroll_dist': 0,
                                                                                         'time_spent': 12},
 'https://www.cnbc.com/stocks/': {'scroll_dist': 9138.181640625,
                                  'time_spent': 12},
 'https://www.express.co.uk/sport/tennis/1275221/Andy-Murray-Rafael-Nadal-tennis-return-virtual-Madrid-Open': {'scroll_dist': 0,
                                                                                                               'time_spent': 16},
 'https://www.google.com/search?pws=0&gl=us&sxsrf=ALeKk03GiwHrVVtTMQH56oP1lQqTCgT7gg%3A1588243544549&ei=WKyqXoiOIcWR9QOdiY7oAg&q=nadal&oq=nadal&gs_lcp=CgZwc3ktYWIQAzIECAAQQzIECAAQQzIECAAQQzIECAAQQzIECAAQQzIECAAQQzIECAAQQzIECAAQQzIECAAQQzIECAAQQzoECAAQRzoCCAA6BQgAEIMBUK0vWJ8zYJU1aABwA3gAgAHvAYgBhgeSAQUwLjQuMZgBAKABAaoBB2d3cy13aXo&sc

In [17]:
# keywords database structure

# keyword_db = {
#     'python' : {
#         'score'    : 255.0, 
#         'urls'     : ['https://docs.python.org/3/tutorial/index.html', ]
#     },
# }

#load keywords database from file

filename = "./data/keyword_db.data"

try:
    with open(filename, 'rb') as f:
        keyword_db = pickle.load(f)
except:
    keyword_db = {}


In [7]:
# new keywords that we need to add as nodes/add edges for 
# in our clustering graph.

# new_keywords_to_cluster = [['a', 'b'], ['c']]
new_keywords_to_cluster = []

In [9]:
# standardize time, scroll_dist and impute missing scroll distances
total_time=0
time_n=0
total_scroll=0
scroll_n=0

for url in history_dict:
    time = history_dict[url]['time_spent']
    scroll = history_dict[url]['scroll_dist']
    
    total_time += time
    time_n += 1
    if scroll>2.0:
        total_scroll += scroll
        scroll_n += 1
        
avg_time = total_time/time_n
avg_scroll = total_scroll/scroll_n

for url in history_dict:
    if history_dict[url]['scroll_dist']<2.0:
        history_dict[url]['scroll_dist'] = avg_scroll * history_dict[url]['time_spent'] / avg_time

In [10]:
def update_score(relevance_feedback):
    score_matrix = {
        'time_score' : 1.0, 
        'scroll_score' : 0.001,
    }
    
    increment_score = (relevance_feedback['time_spent'] * score_matrix['time_score'] +
                              relevance_feedback['scroll_dist'] * score_matrix['scroll_score'])
    
    return increment_score

    

In [18]:
def process_new_history(history_dict):
    
    for url in history_dict:

#         try:
        # taking 5 most significant keywords
        try:
            keywords_list = keywordExtractor(url).items()
            keywords_list = sorted(keywords_list, key=lambda key: key[1])
            keywords_list = keywords_list[:5]
        except:
            continue
        print(url)
        pprint(keywords_list)

        # add keywords to be clustered
        new_keywords_to_cluster.append(
            [word for (word, value) in keywords_list])

        relevance_feedback = history_dict[url]

        # make new entry for keyword in keyword_db 
        # or if already present: 
        #    1) update score
        #    2) add url to the keyword's list of urls

        for (word, value) in keywords_list:
            if word in keyword_db:
                keyword_db[word]['score'] += update_score(relevance_feedback)
                if url not in keyword_db[word]['urls']:
                    keyword_db[word]['urls'].append(url)
            else:
                keyword_db[word] = {}
                keyword_db[word]['score'] = update_score(relevance_feedback)
                keyword_db[word]['urls'] = [url]

                
#         except:
#             continue
    

In [19]:
process_new_history(history_dict)

https://www.google.com/search?q=stocks&pws=0&gl=us&gws_rd=cr
[('stocks', 0.002143040549791458),
 ('stock', 0.0115394491142617),
 ('search', 0.018005731768732075),
 ('market', 0.02536191802406661),
 ('ago', 0.02758474604308135)]
https://www.marketwatch.com/investing/stocks
[('stock', 0.004186046493765303),
 ('apr.', 0.0064266968995946834),
 ('screener', 0.015398049681870566),
 ('updated', 0.015738230767960763),
 ('stocks', 0.01850673186717292)]
https://www.cnbc.com/stocks/
[('apr', 0.0004988187494706606),
 ('wed', 0.002355809049656511),
 ('market', 0.0029947478827743973),
 ('politics', 0.0033609513453103377),
 ('stock', 0.0045382708868108405)]
https://www.investopedia.com/stocks-4427785
[('essentials', 0.004793314122059439),
 ('investing', 0.005590778484115367),
 ('stock', 0.005704315029982178),
 ('trading', 0.008439434249713143),
 ('management', 0.010231523976163245)]
https://www.google.com/search?q=bipartite&pws=0&gl=us&gws_rd=cr
[('bipartite', 0.0016614438464015273),
 ('graph', 0.010

In [ ]:
# print(history_dict)

In [20]:
# save keywords to cluster
with open('./data/to_cluster.data', 'wb') as outfile:
    pickle.dump(new_keywords_to_cluster, outfile)

In [21]:
# save keywords database
with open('./data/keyword_db.data', 'wb') as outfile:
    pickle.dump(keyword_db, outfile)

In [22]:
pprint(keyword_db)

{'ago': {'score': 27.39272705078125,
         'urls': ['https://www.google.com/search?q=stocks&pws=0&gl=us&gws_rd=cr']},
 'andy': {'score': 19.69261762283509,
          'urls': ['https://www.express.co.uk/sport/tennis/1275221/Andy-Murray-Rafael-Nadal-tennis-return-virtual-Madrid-Open']},
 'apr': {'score': 21.138181640625, 'urls': ['https://www.cnbc.com/stocks/']},
 'apr.': {'score': 11.349090820312501,
          'urls': ['https://www.marketwatch.com/investing/stocks']},
 'bipartite': {'score': 46.225826742516944,
               'urls': ['https://www.google.com/search?q=bipartite&pws=0&gl=us&gws_rd=cr',
                        'https://networkx.github.io/documentation/stable/reference/algorithms/bipartite.html',
                        'https://www.whitman.edu/mathematics/cgt_online/book/section05.04.html']},
 'essentials': {'score': 19.69261762283509,
                'urls': ['https://www.investopedia.com/stocks-4427785']},
 'google': {'score': 48.98564060514519,
            'urls': ['